# Sheet properties

# Imports

In [ ]:
import sys
sys.path.append('/home/rcendre/classification')
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"]="1";  
from numpy import array
from sklearn.preprocessing import LabelEncoder
from toolbox.classification.common import Folds, IO, Tools
from toolbox.models.builtin import Applications
from toolbox.transforms.labels import OrderedEncoder
from toolbox.views.common import Views
from toolbox.classification.parameters import Dermatology, Settings

# Parameters

In [ ]:
# Advanced parameters
use_unknown = False 
validation = 4
settings = Settings.get_default_dermatology()

# Inputs

In [ ]:
inputs = Dermatology.images(modality='Microscopy', data_type=data_type, use_unknown=use_unknown)

In [ ]:
# Transform groups
group_encoder = LabelEncoder().fit(array(inputs['ID_Patient'].tolist()))
Tools.transform(inputs, {'datum': 'ID_Patient'}, group_encoder, 'GroupEncode')
# Transform labels
label_encoder = OrderedEncoder().fit(['Normal', 'Benign', 'Malignant'])
Tools.transform(inputs, {'datum': 'Label'}, label_encoder, 'LabelEncode')
# Make folds
Folds.build_group_folds(inputs, {'datum': 'Datum', 'label_encode': 'LabelEncode', 'group': 'GroupEncode'}, validation)

# Feature extraction

In [ ]:
features_file = f'Extraction_Transfer.pickle'

In [ ]:
architectures = [('VGG16', {}), ('InceptionV3', {}), ('InceptionResNetV2', {}), ('ResNet', {})]

In [ ]:
for architecture in architectures:
    architecture_name = architecture[0]
    architecture_additional = architecture[1]
    # Extracting
    method = Applications.get_transfer_learning(architecture=architecture_name, pooling='avg', batch_size=1, additional=architecture_additional)
    method.transform = method.predict_proba
    Tools.transform(inputs, {'datum': 'Datum'}, method, f'{architecture_name}Avg')

    # Extracting
    method = Applications.get_transfer_learning(architecture=architecture_name, pooling='max', batch_size=1, additional=architecture_additional)
    method.transform = method.predict_proba
    Tools.transform(inputs, {'datum': 'Datum'}, method, f'{architecture_name}Max')
    
    # Save    
    IO.save(inputs, features_file)

# Features

In [ ]:
inputs = IO.load(features_file)
for architecture in architectures: 
    extractor_name, extractor_process = architecture
    # PCA + TSNE
    figure = Views.projection(inputs, {'datum': f'{extractor_name}Avg', 'label': 'Label'}, settings);
    figure.savefig(f"{extractor_name}Avg.png", dpi=300)
    
    figure = Views.projection(inputs, {'datum': f'{extractor_name}Max', 'label': 'Label'}, settings);
    figure.savefig(f"{extractor_name}Max.png", dpi=300)